In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!ls '/content/drive/My Drive/CP8318/'

'Hyperparameter testing.ipynb'	        normalized_5_game_average_final.csv
 normalized_10_game_average_final.csv   optuna_test.py
 normalized_15_game_average_final.csv


In [5]:
import warnings

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam, Adamax, Nadam, Adadelta, Adagrad, SGD
from sklearn.model_selection import train_test_split
from keras import regularizers
import optuna
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
import pandas as pd

In [9]:
def objective(trial):
    '''
    TODO: Add suggest categorical for ??
    '''
    # Clear clutter from previous Keras session graphs.
    clear_session()
    data = pd.read_csv('/content/drive/My Drive/CP8318/' + DATA_FILE)
    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
    x, y = data.iloc[:, :-1], data.iloc[:, -1:]
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.9, random_state=42)
    input_shape = x_train.shape[1]
    hidden = trial.suggest_categorical("hidden_size", [i for i in range(1, 1000)])
    num_hidden = trial.suggest_categorical("num_hidden", [i for i in range(1, 6)])
    activation_function = trial.suggest_categorical("activation_function", ["sigmoid", "tanh", "relu"])
    optimizer = trial.suggest_categorical("optimization_algorithm", ["RMSprop", "Adam", "Adamax", "Nadam", "Adadelta", "Adagrad", "SGD"])
    print(hidden)
    print(num_hidden)
    print(optimizer)
    model = Sequential()
    model.add(
        Dense(
            int(hidden),
            activation=activation_function,
            input_shape=(input_shape,),
            use_bias=True,
            name="Input",
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
        )
    )
    if num_hidden > 1:
        for i in range(1, num_hidden):
            model.add(
                Dense(
                    int(hidden),
                    activation=activation_function,
                    use_bias=True,
                    name="HiddenLayer{0}".format(i+1),
                    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                )
            )
    model.add(Dense(1, activation="sigmoid"))

    opt = None
    lr = trial.suggest_float("lr", 1e-50, 1e-1, log=True)
    if optimizer == "RMSprop":
        opt = RMSprop(lr=lr)
    elif optimizer == "Adam":
        opt = Adam(lr=lr)
    elif optimizer == "Adamax":
        opt = Adamax(lr=lr)
    elif optimizer == "Nadam":
        opt = Nadam(lr=lr)
    elif optimizer == "Adadelta":
        opt = Adadelta(lr=lr)
    elif optimizer == "Adagrad":
        opt = Adagrad(lr=lr)
    elif optimizer == "SGD":
        opt = SGD(lr=lr)


    model.compile(
        loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        epochs=100000,
        verbose=False,
        callbacks=[callback]
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    print(score)
    return score[0]

In [ ]:
N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10
DATA_FILE = "normalized_15_game_average_final.csv"
study = optuna.create_study(direction="minimize")
!ls '/content/drive/My Drive/CP8318/'
a = pd.read_csv('/content/drive/My Drive/CP8318/normalized_15_game_average_final.csv')
print(a)
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print(DATA_FILE)
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-11-21 00:29:15,026] A new study created in memory with name: no-name-d02aaca3-662b-4157-b2cf-94dd3e89127c


'Hyperparameter testing.ipynb'	        normalized_5_game_average_final.csv
 normalized_10_game_average_final.csv   optuna_test.py
 normalized_15_game_average_final.csv
      Unnamed: 0   HteamFG   HteamTS  ...  AteamEDiff     Awin%  target
0              0  0.002989  0.003040  ...    0.052902  0.006056       0
1              1  0.002660  0.003150  ...   -0.066880  0.000000       1
2              2  0.002706  0.003147  ...   -0.066755  0.000000       1
3              3  0.002285  0.002790  ...    0.013763  0.006301       0
4              4  0.002461  0.003108  ...   -0.066493  0.000000       1
...          ...       ...       ...  ...         ...       ...     ...
7374        7374  0.002403  0.002867  ...   -0.025567  0.001918       1
7375        7375  0.002420  0.002912  ...    0.023370  0.003496       1
7376        7376  0.002533  0.003094  ...   -0.031278  0.001374       0
7377        7377  0.002448  0.003009  ...   -0.012146  0.002564       1
7378        7378  0.002650  0.003079  ..

[I 2020-11-21 00:29:35,785] Trial 0 finished with value: 1.6257338523864746 and parameters: {'hidden_size': 708, 'num_hidden': 5, 'activation_function': 'sigmoid', 'optimization_algorithm': 'Adam', 'lr': 7.543586673222383e-43}. Best is trial 0 with value: 1.6257338523864746.


[1.6257338523864746, 0.5783365368843079]
289
5
RMSprop


[I 2020-11-21 00:29:42,726] Trial 1 finished with value: 0.9829469919204712 and parameters: {'hidden_size': 289, 'num_hidden': 5, 'activation_function': 'relu', 'optimization_algorithm': 'RMSprop', 'lr': 3.2322594216672284e-41}. Best is trial 1 with value: 0.9829469919204712.


...

[0.6506389379501343, 0.6170212626457214]
87
1
Adagrad


[I 2020-11-21 00:35:30,422] Trial 13 finished with value: 0.6457276940345764 and parameters: {'hidden_size': 87, 'num_hidden': 1, 'activation_function': 'sigmoid', 'optimization_algorithm': 'Adagrad', 'lr': 0.09175918422005265}. Best is trial 13 with value: 0.6457276940345764.


[0.6457276940345764, 0.6170212626457214]
209
2
Adagrad


[I 2020-11-21 00:35:34,977] Trial 14 finished with value: 0.8107661604881287 and parameters: {'hidden_size': 209, 'num_hidden': 2, 'activation_function': 'sigmoid', 'optimization_algorithm': 'Adagrad', 'lr': 8.391499275000712e-10}. Best is trial 13 with value: 0.6457276940345764.


[0.8107661604881287, 0.5783365368843079]
411
1
Adamax
